In [17]:
import pyspark.sql.functions as F
import delta

In [18]:
# rawfolder = "abfss://raw@dobbinsdatalake.dfs.core.windows.net/weatherbit/*.json"
# schema_file = "abfss://raw@dobbinsdatalake.dfs.core.windows.net/weatherbit/2023_07_21_16_00_23_UTC.json"
silver_folder = "abfss://silver@dobbinsdatalake.dfs.core.windows.net/delta/weather_silver/weather"
gold_folder = "abfss://gold@dobbinsdatalake.dfs.core.windows.net/delta/weather_gold/weather"

In [19]:
# aggregate from weather_silver.weather table (this goes back 30 days so we don't aggregate the entire silver layer)

df = spark.sql(f"""
    with readings as (
        SELECT date_trunc('DAY',timestampadd(HOUR, -6, to_timestamp(ob_time))) AS ObDateST,
            double((1.8 * temp) + 32) as AirTemp,
            double(gust) * 2.237 as gust --mph
        from weather_silver.weather
        where date_trunc('DAY',timestampadd(HOUR, -6, to_timestamp(ob_time))) >= date_add(current_timestamp(), -30)
    )
    select ObDateST, 
        round(min(AirTemp),1) as min_temp,
        round(max(AirTemp),1) as max_temp,
        round(avg(AirTemp),1) as avg_temp,
        round(max(gust)) as max_gust
    from readings
    group by ObDateST""")

In [20]:
# initial write
# df.write.format("delta").mode("overwrite").save(gold_folder)

In [21]:
delta_df = delta.DeltaTable.forPath(spark, gold_folder)

(delta_df
    .alias("delta_wx")
    .merge(df.alias("updated_wx"),
        "delta_wx.ObDateST == updated_wx.ObDateST")
        .whenMatchedUpdate(set = {"ObDateST":"updated_wx.ObDateST", "min_temp":"updated_wx.min_temp", "max_temp":"updated_wx.max_temp", "avg_temp":"updated_wx.avg_temp", "max_gust":"updated_wx.max_gust"})
        .whenNotMatchedInsert(values = {"ObDateST":"updated_wx.ObDateST", "min_temp":"updated_wx.min_temp", "max_temp":"updated_wx.max_temp", "avg_temp":"updated_wx.avg_temp", "max_gust":"updated_wx.max_gust"})
        .execute()
)

In [22]:
%%sql

create database if not exists weather_gold;

In [23]:
%%sql

create table if not exists weather_gold.weather_obs_summary
using delta
location 'abfss://gold@dobbinsdatalake.dfs.core.windows.net/delta/weather_gold/weather'

In [24]:
# %%sql

# describe history weather_gold.weather_obs_summary

In [25]:
# version as ints or longs
# df_changes = spark.read.format("delta") \
#   .option("readChangeFeed", "true") \
#   .option("startingVersion", 19) \
#   .option("endingVersion", 22) \
#   .table("weather_gold.weather_obs_summary")

In [26]:
# display(df_changes.sort(F.col("ObDateST").desc()))
# df_changes.createOrReplaceTempView("changes")